In [21]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import random

def next_batch(num_split, feature, labels):
    ind = np.arange(0, len(feature))
    np.random.shuffle(ind)
    ind = ind[:num_split]
    feature_shuffle = [feature[i].flatten() for i in ind]
    labels_shuffle = [labels[i] for i in ind]

    return np.asarray(feature_shuffle), np.asarray(labels_shuffle)

x_train = np.load("../saved_data/x_train.npy", allow_pickle=True)
x_test = np.load("../saved_data/x_test.npy", allow_pickle=True)
y_train = np.load("../saved_data/y_train.npy", allow_pickle=True)
y_test = np.load("../saved_data/y_test.npy", allow_pickle=True)

# Parameters
learning_rate = 0.0005
training_epochs = 150
batch_size = 256
display_step = 10

x = tf.placeholder(tf.float32, [None, 22500])
y = tf.placeholder(tf.float32, [None, 2])

W1 = tf.Variable(tf.random_normal([22500, 256], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([256]), name='b1')
z1 = tf.nn.relu(tf.matmul(x,W1)+b1)

W2 = tf.Variable(tf.random_normal([256, 128], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([128]), name='b2')
z2 = tf.nn.relu(tf.matmul(z1,W2)+b2)

W3 = tf.Variable(tf.random_normal([128, 64], stddev=0.03), name='W3')
b3 = tf.Variable(tf.random_normal([64]), name='b3')
z3 = tf.nn.relu(tf.matmul(z2,W3)+b3)

W4 = tf.Variable(tf.random_normal([64, 2], stddev=0.03), name='W4')
b4 = tf.Variable(tf.random_normal([2]), name='b4')


log = tf.matmul(z3, W4) + b4
pred = tf.nn.softmax(tf.matmul(z3, W4) + b4)

# pred = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W4), b4))

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=log))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# acc_ = tf.summary.scalar('acc', accuracy)
# loss_summary = tf.summary.scalar('loss', cost)


with tf.Session() as sess:
    #writer = tf.summary.FileWriter('../logs/NeuralNetwork', sess.graph)
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(x_train)/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = next_batch(batch_size, x_train, y_train)
            _, c, acc = sess.run([optimizer, cost, accuracy], feed_dict={x: batch_xs, y: batch_ys})
            avg_cost += c / total_batch
        if (epoch+1) % display_step == 0:
#             writer.add_summary(acc_summary, epoch)
#             writer.add_summary(loss_, epoch)
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost), "acc=", "{:.5}".format(acc))
    #writer.close()
    print("Optimization Finished!")

    #saver = tf.train.Saver()
    #saver.save(sess, "../model/NeuralNetwork/model")
    
    x_test_flat = []
    for i in x_test:
        x_test_flat.append(i.flatten())
    x_test_flat = np.array(x_test_flat)
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: x_test_flat, y: y_test}))

Epoch: 0001 cost= 2.160758722 acc= 0.65625
Epoch: 0002 cost= 0.397952881 acc= 0.82031
Epoch: 0003 cost= 0.383790351 acc= 0.875
Epoch: 0004 cost= 0.337312444 acc= 0.82812
Epoch: 0005 cost= 0.337649010 acc= 0.91406
Epoch: 0006 cost= 0.268956622 acc= 0.9375
Epoch: 0007 cost= 0.314791990 acc= 0.94531
Epoch: 0008 cost= 0.269474661 acc= 0.92969
Epoch: 0009 cost= 0.187832513 acc= 0.92188
Epoch: 0010 cost= 0.292114638 acc= 0.92969
Epoch: 0011 cost= 0.203120043 acc= 0.89844
Epoch: 0012 cost= 0.217981219 acc= 0.92188
Epoch: 0013 cost= 0.184234965 acc= 0.97656
Epoch: 0014 cost= 0.238432910 acc= 0.96094
Epoch: 0015 cost= 0.148643354 acc= 0.92969
Epoch: 0016 cost= 0.149543560 acc= 0.9375
Epoch: 0017 cost= 0.239602813 acc= 0.92188
Epoch: 0018 cost= 0.161043364 acc= 0.94531
Epoch: 0019 cost= 0.296214859 acc= 0.92188
Epoch: 0020 cost= 0.130897954 acc= 0.95312
Epoch: 0021 cost= 0.191322305 acc= 0.95312
Epoch: 0022 cost= 0.128030331 acc= 0.92188
Epoch: 0023 cost= 0.135070244 acc= 0.91406
Epoch: 0024 cos